In [2]:
import pandas as pd

In [6]:
# Lire les donées
df = pd.read_parquet('/workspaces/mlops-zoomcamp/01-intro/data/yellow_tripdata_2023-01.parquet')
df 

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [7]:
# Nombre de colonnes
num_columns = df.shape[1]
print(f"Q1. Le nombre de colonnes est : {num_columns}")

Q1. Le nombre de colonnes est : 19


In [8]:
# Convertir les colonnes de temps en datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

# Calculer la durée du trajet en minutes
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60

# Écart-type de la durée
std_duration = df['duration'].std()
print(f"Q2. L'écart-type de la durée est : {std_duration:.2f} minutes")

Q2. L'écart-type de la durée est : 42.59 minutes


In [10]:
# Supprimer les outliers
df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

# Fraction des enregistrements récents
fraction_remaining = len(df) / len(pd.read_parquet('/workspaces/mlops-zoomcamp/01-intro/data/yellow_tripdata_2023-01.parquet'))
print(f"Q3. La fraction des enregistrements restants est : {fraction_remaining * 100:.0f}%")

Q3. La fraction des enregistrements restants est : 98%


In [11]:
from sklearn.feature_extraction import DictVectorizer

In [12]:
# Convertir le DataFrame en liste de dictionnaires
categorial = ['PULocationID', 'DOLocationID']
df[categorial] = df[categorial].astype(str)

train_dicts = df[categorial].to_dict(orient='records')

# Fit le DictVectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Dimensionalité de la matrice
dimensionality = X_train.shape[1]
print(f"Q4. La dimensionalité de la matrice est : {dimensionality}")

Q4. La dimensionalité de la matrice est : 515


In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [14]:
# Entraîner le modèle
model = LinearRegression()
model.fit(X_train, df['duration'])

# Prédiction
y_pred = model.predict(X_train)

# RMSE
mse = mean_squared_error(df['duration'], y_pred)
rmse = np.sqrt(mse)
print(f"Q5. Le RMSE du modèle est : {rmse:.2f}")

Q5. Le RMSE du modèle est : 7.65


In [15]:
# Lire les données de validation
df_val = pd.read_parquet('/workspaces/mlops-zoomcamp/01-intro/data/yellow_tripdata_2023-02.parquet')

# Préparer les données de validation
df_val['tpep_pickup_datetime'] = pd.to_datetime(df_val['tpep_pickup_datetime'])
df_val['tpep_dropoff_datetime'] = pd.to_datetime(df_val['tpep_dropoff_datetime'])

df_val['duration'] = (df_val['tpep_dropoff_datetime'] - df_val['tpep_pickup_datetime']).dt.total_seconds() / 60
df_val = df_val[(df_val['duration'] >= 1) & (df_val['duration'] <= 60)]
df_val[categorial] = df_val[categorial].astype(str)
val_dicts = df_val[categorial].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Prédiction
y_val_pred = model.predict(X_val)

# RMSE
mse_val = mean_squared_error(df_val['duration'], y_val_pred)
rmse_val = np.sqrt(mse_val)
print(f"Q6. Le RMSE du modèle sur les données de validation est : {rmse_val:.2f}")

Q6. Le RMSE du modèle sur les données de validation est : 7.81
